In [1]:
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

# Test the type of distribution of data for any distribution
from scipy.stats import kstest
# Test if data is distributed in normal way
from scipy.stats import shapiro
# Test with parametric t-test
from scipy.stats import ttest_ind, ttest_rel
# Test with nonparametric test for comparing of two
from scipy.stats import wilcoxon
# Test with nonparametric test for comparing of two or more
from scipy.stats import kruskal
# Test with nonparametric test for comparing of three or more
from scipy.stats import friedmanchisquare
# probability plot
from scipy.stats import probplot
# data preparation
from sklearn.preprocessing import normalize

np.set_printoptions(linewidth=100000000, formatter={'all': lambda x: str(x)})
%matplotlib inline

In [2]:
dims, algos = [10, 30, 50], ['dynNpMsjDE', 'DE', 'jDE', 'BBFWA', 'SCA', 'ES(1+1)', 'dynFWAG', 'ASO', 'BA', 'MTS']

# CEC 2017 algorithm speed test
Example of time execution speed of algorithm

In [ ]:
sys.path.append('cec2017')
from cec2017 import run_fun
from run_cec import MinMB
from NiaPy import Runner
from NiaPy.util import Task

def testOne(x=0.55):
   for i in range(10 ** 6): 
      x = x + x
      x = x / 2
      x = x * x
      x = np.sqrt(x)
      x = np.log(x)
      x = np.exp(x)
      x = x / (x + 2)

def testTwo(d):
   for i in range(2 * 10 ** 5): 
      x = np.random.uniform(-100, 100, d)
      run_fun(x, 18)

def runTree(a, d):
   task = Task(D=d, nFES=2 * 10 ** 5, benchmark=MinMB(run_fun, fnum=18))
   algo = Runner.getAlgorithm(a)(task=task)
   start_time = time.time()
   algo.run()
   return time.time() - start_time

In [ ]:
start_time = time.time()
testOne()
t0 = (time.time() - start_time)

print ('t0: ', t0)

t1 = []
for d in dims:
   start_time = time.time()
   testTwo(d)
   t1.append(time.time() - start_time)
   
print ('t1: ', t1)

t2 = []
for d in dims:
   tmp = []
   for a in algos:   
      r = np.full(5, 0.0)
      for i in range(len(r)): r[i] = runTree(a, d)
      tmp.append(np.mean(r))
   t2.append(tmp)

for i, d in enumerate(dims):
   for j, a in enumerate(algos):
      print ('%10s %d -- %.4E' % (a, d, (t2[i][j] - t1[i]) / t0))

# CEC 2017 statistic
For getting the data need to perform statistic test we used: `for i in {1..30}; do python run_cec.py -c 17 -o T -rn 50 -a DE -D 10 -f $i -seed {1000..1050}; done`.
The example shows how to run DE algorithm on all benchmark functions that have problem dimensionality set to 10.
DE algorithm runs 50 times on each benchmark functions with seed in range from 1000 to 1050.
Every algorithm run has it's own seed.
We used `-o T` for generating the output.

## Example of multiple runs on one problem
* dim $\in \{10, 30, 50\}$
* fnum $\in \{1, \cdots , 30\}$
* algos $\in$ `{dynNpMsjDE, BBFWA, DE, jDE, SCA, ES(1+1), ES(m+1), ASO, BA, dynFWAG}`

## Load data
We will create a new variable `data`, that holds the optization results for 50 runs, 30 functions and specified algorithms.
Now first index presents algorithm, second index presents function and third index presents the value of optimization.

In [3]:
dim, data = dims[2], []
data = np.asanyarray([[np.loadtxt('data/%s_%d_%d_v' % (a, fnum, dim)) for fnum in range(1, 31)] for a in algos])
# Get basic statistics
vals = []
for fnum in range(30):
   tmp = []
   print ('\nfun_num: %d' % (fnum + 1))
   for i, a in enumerate(algos):
      d = data[i, fnum] - (fnum + 1) * 100
      print ('%10s:\tmin: %.3E \tmean: %.3E\tstd: %.3E' % (a, np.min(d), np.mean(d), np.std(d)))
      tmp.append((np.min(d), np.mean(d), np.std(d)))
   vals.append(tmp)
vals = np.asanyarray(vals)
# Get best values for basic statistics
imin, imean, istd = [], [], []
for fnum in range(30):
   imin.append(np.argmin([vals[fnum, i, 0] for i in range(len(algos))]))
   imean.append(np.argmin([vals[fnum, i, 1] for i in range(len(algos))]))
   istd.append(np.argmin([vals[fnum, i, 2] for i in range(len(algos))]))
# Generate table entrys for latex
out = ''
for i in range(len(algos)):
   for fnum in range(30): out += ('%.3E' if i != imin[fnum] else '\\textbf{%.3E}') % vals[fnum, i, 0] + ' & ' + ('%.3E' if i != imean[fnum] else '\\textbf{%.3E}') % vals[fnum, i, 1] + ' & ' + ('%.3E' if i != istd[fnum] else '\\textbf{%.3E}') % vals[fnum, i, 2] + ' \\\\ \n'
   out += '\n'
print ('\n', out)


fun_num: 1
dynNpMsjDE:	min: 7.218E-11 	mean: 3.117E-01	std: 2.099E+00
        DE:	min: 4.422E+09 	mean: 7.078E+09	std: 9.173E+08
       jDE:	min: 7.367E+09 	mean: 9.988E+09	std: 1.068E+09
     BBFWA:	min: 5.899E+01 	mean: 2.977E+04	std: 3.884E+04
       SCA:	min: 1.381E+10 	mean: 2.755E+10	std: 5.091E+09
   ES(1+1):	min: 3.130E-01 	mean: 5.076E+03	std: 7.389E+03
   dynFWAG:	min: 4.889E+09 	mean: 4.768E+10	std: 1.792E+10
       ASO:	min: 2.595E+04 	mean: 7.467E+07	std: 2.273E+08
        BA:	min: 4.189E+10 	mean: 9.936E+10	std: 2.710E+10
       MTS:	min: 1.752E+00 	mean: 8.802E+01	std: 1.254E+02

fun_num: 2
dynNpMsjDE:	min: 8.579E-07 	mean: 3.091E+01	std: 4.134E+01
        DE:	min: 1.368E+52 	mean: 7.392E+54	std: 1.751E+55
       jDE:	min: 2.105E+51 	mean: 2.601E+55	std: 8.653E+55
     BBFWA:	min: 6.212E-03 	mean: 1.767E-02	std: 6.134E-03
       SCA:	min: 5.955E+53 	mean: 3.824E+65	std: 2.260E+66
   ES(1+1):	min: 8.828E-03 	mean: 1.498E+05	std: 8.164E+05
   dynFWAG:	min: 1.653E+45 	mean

## Normalize the data
Now we transform the presentation of data in our new array called `ndata` that has normalized data.
Now first index presents function, second index presents algorithm and third index presents the value of optimization.

In [4]:
ndata = np.asanyarray([normalize([data[j][i] for j in range(len(algos))]) for i in range(30)])

## Test for normal distribution
[Kolmogorov–Smirnov](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) test: Tests whether a sample is drawn from a given distribution, or whether two samples are drawn from the same distribution.

If $\text{p-value} > \alpha$ then values belong to selected distribution.

In [5]:
for i, a in enumerate(algos):
   for j in range(30):
      print(a, ' on ', j + 1, ' ', kstest(ndata[j, i], 'norm'))

dynNpMsjDE  on  1   KstestResult(statistic=0.5560456991173068, pvalue=4.440892098500626e-15)
dynNpMsjDE  on  2   KstestResult(statistic=0.5479862274166433, pvalue=1.2878587085651816e-14)
dynNpMsjDE  on  3   KstestResult(statistic=0.5562313465107563, pvalue=4.440892098500626e-15)
dynNpMsjDE  on  4   KstestResult(statistic=0.5475923366953921, pvalue=1.354472090042691e-14)
dynNpMsjDE  on  5   KstestResult(statistic=0.5490458964961522, pvalue=1.1102230246251565e-14)
dynNpMsjDE  on  6   KstestResult(statistic=0.5560125493334245, pvalue=4.440892098500626e-15)
dynNpMsjDE  on  7   KstestResult(statistic=0.5476917496818939, pvalue=1.3322676295501878e-14)
dynNpMsjDE  on  8   KstestResult(statistic=0.5524298718169383, pvalue=7.105427357601002e-15)
dynNpMsjDE  on  9   KstestResult(statistic=0.5323391138252415, pvalue=9.303668946358812e-14)
dynNpMsjDE  on  10   KstestResult(statistic=0.5433212707473006, pvalue=2.3314683517128287e-14)
dynNpMsjDE  on  11   KstestResult(statistic=0.5534007089552706, p

### Plot

In [ ]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html
ax1 = plt.subplot(221)
probplot(ndata[i][0], plot=plt)

[Shapiro–Wilk test](https://en.wikipedia.org/wiki/Shapiro%E2%80%93Wilk_test) is a test of normality in frequentist statistics.

In [7]:
for i, a in enumerate(algos):
   for j in range(30):
      print(a, ' on ', j + 1, ' ', shapiro(ndata[j, i]))

dynNpMsjDE  on  1   (0.1320023536682129, 1.2831264837196376e-15)
dynNpMsjDE  on  2   (0.76045823097229, 1.204191590886694e-07)
dynNpMsjDE  on  3   (0.25364434719085693, 1.6197638255622746e-14)
dynNpMsjDE  on  4   (0.9263989329338074, 0.004048010800033808)
dynNpMsjDE  on  5   (0.9811744093894958, 0.6024453639984131)
dynNpMsjDE  on  6   (0.890066921710968, 0.0002294408914167434)
dynNpMsjDE  on  7   (0.9702857732772827, 0.2377147227525711)
dynNpMsjDE  on  8   (0.9801079630851746, 0.5565255880355835)
dynNpMsjDE  on  9   (0.9253937005996704, 0.003714574733749032)
dynNpMsjDE  on  10   (0.9844216108322144, 0.7464805841445923)
dynNpMsjDE  on  11   (0.9491251707077026, 0.031408995389938354)
dynNpMsjDE  on  12   (0.9259539246559143, 0.0038966615684330463)
dynNpMsjDE  on  13   (0.6927800178527832, 6.155380649630615e-09)
dynNpMsjDE  on  14   (0.8006125688552856, 9.057126248990244e-07)
dynNpMsjDE  on  15   (0.8250914812088013, 3.4940130717586726e-06)
dynNpMsjDE  on  16   (0.9398443698883057, 0.0132

/home/klemen/.local/share/virtualenvs/CEC-jhcRP2Wl/lib/python3.7/site-packages/scipy/stats/morestats.py:1306: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


## Run t-test related

In [13]:
for i, a in enumerate(algos):
   for j in range(30):
      print ('%10s vs. %10s on %d. fun:  %s' % (algos[0], a, j, ttest_rel(ndata[j, 0], ndata[j, i])))

dynNpMsjDE vs. dynNpMsjDE on 0. fun:  Ttest_relResult(statistic=nan, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 1. fun:  Ttest_relResult(statistic=nan, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 2. fun:  Ttest_relResult(statistic=nan, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 3. fun:  Ttest_relResult(statistic=nan, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 4. fun:  Ttest_relResult(statistic=nan, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 5. fun:  Ttest_relResult(statistic=nan, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 6. fun:  Ttest_relResult(statistic=nan, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 7. fun:  Ttest_relResult(statistic=nan, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 8. fun:  Ttest_relResult(statistic=nan, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 9. fun:  Ttest_relResult(statistic=nan, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 10. fun:  Ttest_relResult(statistic=nan, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 11. fun:  Ttest_relResult(statistic=nan, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE 

## Run Wilcoxon test
[Wilcoxon signed-rank](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test) test: tests whether matched pair samples are drawn from populations with different mean ranks

In [14]:
for i, a in enumerate(algos):
   for j in range(30):
      print ('%10s vs. %10s on %d. fun:  %s' % (algos[0], a, j, wilcoxon(ndata[j, 0], ndata[j, i])))

dynNpMsjDE vs. dynNpMsjDE on 0. fun:  WilcoxonResult(statistic=0.0, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 1. fun:  WilcoxonResult(statistic=0.0, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 2. fun:  WilcoxonResult(statistic=0.0, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 3. fun:  WilcoxonResult(statistic=0.0, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 4. fun:  WilcoxonResult(statistic=0.0, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 5. fun:  WilcoxonResult(statistic=0.0, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 6. fun:  WilcoxonResult(statistic=0.0, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 7. fun:  WilcoxonResult(statistic=0.0, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 8. fun:  WilcoxonResult(statistic=0.0, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 9. fun:  WilcoxonResult(statistic=0.0, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 10. fun:  WilcoxonResult(statistic=0.0, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 11. fun:  WilcoxonResult(statistic=0.0, pvalue=nan)
dynNpMsjDE vs. dynNpMsjDE on 12. fun: 

/home/klemen/.local/share/virtualenvs/CEC-jhcRP2Wl/lib/python3.7/site-packages/scipy/stats/morestats.py:2388: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
/home/klemen/.local/share/virtualenvs/CEC-jhcRP2Wl/lib/python3.7/site-packages/scipy/stats/morestats.py:2413: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se


## Run Kruskal test
[Kruskal–Wallis one-way analysis of variance by ranks](https://en.wikipedia.org/wiki/Kruskal%E2%80%93Wallis_one-way_analysis_of_variance): tests whether > 2 independent samples are drawn from the same distribution

In [15]:
for i, a in enumerate(algos):
   for j in range(30):
      print ('%10s vs. %10s on %d. fun:  %s' % (algos[0], a, j, kruskal(ndata[j, 0], ndata[j, i])))

dynNpMsjDE vs. dynNpMsjDE on 0. fun:  KruskalResult(statistic=0.0, pvalue=1.0)
dynNpMsjDE vs. dynNpMsjDE on 1. fun:  KruskalResult(statistic=0.0, pvalue=1.0)
dynNpMsjDE vs. dynNpMsjDE on 2. fun:  KruskalResult(statistic=0.0, pvalue=1.0)
dynNpMsjDE vs. dynNpMsjDE on 3. fun:  KruskalResult(statistic=0.0, pvalue=1.0)
dynNpMsjDE vs. dynNpMsjDE on 4. fun:  KruskalResult(statistic=0.0, pvalue=1.0)
dynNpMsjDE vs. dynNpMsjDE on 5. fun:  KruskalResult(statistic=0.0, pvalue=1.0)
dynNpMsjDE vs. dynNpMsjDE on 6. fun:  KruskalResult(statistic=0.0, pvalue=1.0)
dynNpMsjDE vs. dynNpMsjDE on 7. fun:  KruskalResult(statistic=0.0, pvalue=1.0)
dynNpMsjDE vs. dynNpMsjDE on 8. fun:  KruskalResult(statistic=0.0, pvalue=1.0)
dynNpMsjDE vs. dynNpMsjDE on 9. fun:  KruskalResult(statistic=0.0, pvalue=1.0)
dynNpMsjDE vs. dynNpMsjDE on 10. fun:  KruskalResult(statistic=0.0, pvalue=1.0)
dynNpMsjDE vs. dynNpMsjDE on 11. fun:  KruskalResult(statistic=0.0, pvalue=1.0)
dynNpMsjDE vs. dynNpMsjDE on 12. fun:  KruskalResu

## Run Friedman test
[Friedman two-way analysis of variance by ranks](https://en.wikipedia.org/wiki/Friedman_test): tests whether k treatments in randomized block designs have identical effects

In [20]:
for j in range(30):
   print (algos, ' on ', j + 1, ' fun: ', friedmanchisquare(*ndata[j, :]))

['dynNpMsjDE', 'DE', 'jDE', 'BBFWA', 'SCA', 'ES(1+1)', 'dynFWAG', 'ASO', 'BA', 'MTS']  on  1  fun:  FriedmanchisquareResult(statistic=147.90109090909095, pvalue=2.3993652048924892e-27)
['dynNpMsjDE', 'DE', 'jDE', 'BBFWA', 'SCA', 'ES(1+1)', 'dynFWAG', 'ASO', 'BA', 'MTS']  on  2  fun:  FriedmanchisquareResult(statistic=255.1549090909091, pvalue=8.135119150274281e-50)
['dynNpMsjDE', 'DE', 'jDE', 'BBFWA', 'SCA', 'ES(1+1)', 'dynFWAG', 'ASO', 'BA', 'MTS']  on  3  fun:  FriedmanchisquareResult(statistic=16.455272727272813, pvalue=0.05796183880461555)
['dynNpMsjDE', 'DE', 'jDE', 'BBFWA', 'SCA', 'ES(1+1)', 'dynFWAG', 'ASO', 'BA', 'MTS']  on  4  fun:  FriedmanchisquareResult(statistic=6.240000000000009, pvalue=0.7156785016107658)
['dynNpMsjDE', 'DE', 'jDE', 'BBFWA', 'SCA', 'ES(1+1)', 'dynFWAG', 'ASO', 'BA', 'MTS']  on  5  fun:  FriedmanchisquareResult(statistic=6.615272727272895, pvalue=0.6771046519645224)
['dynNpMsjDE', 'DE', 'jDE', 'BBFWA', 'SCA', 'ES(1+1)', 'dynFWAG', 'ASO', 'BA', 'MTS']  on 